# Exercise 2.5 Stippling (cartopy)
prepared by M.Hauser

Stippling in pyplot is done with the function `contourf`.

> Most of what we show here for georeferenced plots also applies to stippling when you are not using cartopy.

## Import libraries

In [ ]:
import cartopy.crs as ccrs
import cartopy.util as cutil
import matplotlib.pyplot as plt
import mplotutils as mpu
import numpy as np
import xarray as xr

## stippling $\rightarrow$ hatching

Stippling is called hatching in matplotlib.

* `contourf` takes a `hatches` keyword, and you have to specify one hatch-pattern per drawn level. The pattern are determined via characters, e.g. using `"/"` yields diagonal lines.
* The hatching is more dense if the character is repeated, e.g.: `"///"`.
* Specifying an empty string (`""`) omits the hatching.

In [ ]:
# create sample data
lon, lat, data = mpu.sample_data_map(90, 48)
DATA, LON = cutil.add_cyclic_point(data, lon)

# ====

f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.PlateCarree()))
ax.coastlines()

# ====

# add 3 levels
levels = [-1, -0.5, 0.5, 1]

# set hatching for each level
hatches = ["/", "", "////"]

h = ax.contourf(LON, lat, DATA, levels=levels, hatches=hatches, cmap="RdBu")

# ====

ax.set_global()

# add colorbar
mpu.colorbar(h, ax, aspect=15, pad=0.025)

f.canvas.draw()

## Hatch patterns

Most of the hatch patterns are quite intuitive:

```python
patterns = [ "/" , "\\" , "|" , "-" , "+" , "x", "o", "O", ".", "*" ]
```
    
They are visualized in the next figure (after an [stackoverflow answer](https://stackoverflow.com/a/14279608)). Not only `contourf`, other functions also take a hatch keyword, e.g. `ax.bar`. 

(However, `pcolormesh` does not, although the documentation says so.)

In [ ]:
patterns = ["/", "\\", "|", "-", "+", "x", "o", "O", ".", "*", "/."]

f, ax = plt.subplots()
for i, pattern in enumerate(patterns):
    ax.bar(i, 1, color="none", edgecolor="k", hatch=pattern)

Note that \ is an escape character. Therefore, `"\"` is not a valid string. You can either escape the escape character, using `"\\"` or add a space `"\ "`.

## Load CMIP5 data: historical precipitation climatology (1986 to 2005)

Load historical, and projected climatological precipitation, as well as the relative change between them, from all CMIP5 models for RCP8.5 (Taylor et al., 2012).

The data was prepared in [another notebook](../data/prepare_CMIP5_map.ipynb).

In [ ]:
file = "../data/cmip5_delta_pr_rcp85_map.nc"
pr = xr.open_dataset(file)

### Exercise
 * Add hatches for precipitation changes larger than +- 20 % - there are a total of 8 levels (don't change `levels`)

In [ ]:
# get data
pr_rel_cyclic = mpu.cyclic_dataarray(pr.pr_rel)

# ===

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# set levels
levels = np.arange(-20, 101, 20)

# add hatches
# hatches = [...]

h = pr_rel_cyclic.plot.contourf(
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=levels,
    extend="both",
    add_colorbar=False,
    cmap="viridis",
    #     hatches=hatches,
)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional", aspect=15)

plt.draw()

### Solution

In [ ]:
# get data
pr_rel_cyclic = mpu.cyclic_dataarray(pr.pr_rel)

# ===

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# set levels
levels = np.arange(-20, 101, 20)

# add hatches
hatches = ["\\\\", "", "", "//", "//", "//", "//", "//"]

h = pr_rel_cyclic.plot.contourf(
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=levels,
    extend="both",
    add_colorbar=False,
    cmap="viridis",
    hatches=hatches,
)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional", aspect=15)

plt.draw()

### Better Solution

When you look closely, you can see that some of the hatch-lines are broken. It's actually better if you add a second contourf command, with only 3 levels, so that the entire area gets hatched in one go. You can then remove the colored patches with `colors='none'`:

In [ ]:
# get data
pr_rel_cyclic = mpu.cyclic_dataarray(pr.pr_rel)

levels = np.arange(-20, 101, 20)

# ===

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot the filled contours
h = pr_rel_cyclic.plot.contourf(
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=levels,
    extend="both",
    add_colorbar=False,
    cmap="viridis",
)

# add the hatching
levels = [-20, 20]
hatches = ["\\\\", "", "//"]

pr_rel_cyclic.plot.contourf(
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=levels,
    hatches=hatches,
    extend="both",
    add_colorbar=False,
    colors="none",
)


# add colorbar
mpu.colorbar(h, ax, spacing="proportional", aspect=15)

plt.draw()

## Significance hatching

Often we don't want to hatch the same values that we color, but, e.g. stipple significant parts of the data. I recommend to do this in five steps:

1. Plot contourf of the significance values
1. Determine the levels (e.g. `[0, 0.1, 1]`)
1. Add the hatches (e.g. `['...', '']`)
1. Remove the colors with `colors='none'` & get rid of the colorbar
1. Add the actual data you want to plot

### Loading Data

For this example we use data contributed by a course participant. 

The data are ranked probability skill scores (RPSS) of ECMWF system 4 seasonal tercile forecasts verified against ERA-Interim on a global 1° x 1° grid. Additionally, the dataset contains the variable `signif` that indicates grid points with a significant improvement in skill. `signif` is given as boolean, 0 means no improvement, 1 means an improvement.

> Due to the resolution of the data the plotting will take a moment.

In [ ]:
file = "../data/globalRPSS.nc"

ds = xr.open_dataset(file)
ds

In [ ]:
# add cyclic point
signif_cyclic = mpu.cyclic_dataarray(ds.signif, coord="longitude")

opt = dict(transform=ccrs.PlateCarree(), add_colorbar=False)

# =====

f, axs = plt.subplots(
    2, 2, subplot_kw=dict(projection=ccrs.PlateCarree()), gridspec_kw=dict(wspace=0.4)
)

f.set_size_inches(20 / 2.54, 12 / 2.54)
axs = axs.flatten()

# =====

ax = axs[0]
h = signif_cyclic.plot.contourf(ax=ax, **opt)
mpu.colorbar(h, ax, aspect=10)
ax.set_title("Step 1: raw contourf")

# =====

ax = axs[1]
levels = [0, 0.5, 1]
h = signif_cyclic.plot.contourf(ax=ax, levels=levels, **opt)
mpu.colorbar(h, ax, aspect=10)
ax.set_title("Step 2: manual levels")

# =====

ax = axs[2]
levels = [0, 0.5, 1]
hatches = ["", "..."]
h = signif_cyclic.plot.contourf(ax=ax, levels=levels, hatches=hatches, **opt)
mpu.colorbar(h, ax, aspect=10)
ax.set_title("Step 3: hatching")

# =====

ax = axs[3]
levels = [0, 0.5, 1]
hatches = ["", "..."]
h = signif_cyclic.plot.contourf(
    ax=ax, levels=levels, hatches=hatches, colors="none", **opt
)

ax.set_title("Step 4: remove color & bar")

# =====

for ax in axs:
    ax.coastlines()

I would not actually do a four-panel plot. But repeat the same plot over and over, doing one step after the other.

### Final figure

In [ ]:
# add cyclic point
signif_cyclic = mpu.cyclic_dataarray(ds.signif, coord="longitude")

# ========================================

f, ax = plt.subplots(subplot_kw=dict(projection=ccrs.PlateCarree()))
f.subplots_adjust(left=0.1, right=0.8)
ax.coastlines()

# =====

# plot RPSS data

levels = np.arange(-1.0, 1.1, 0.25)

h = ds.RPSS.plot(
    ax=ax,
    levels=levels,
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
    cmap="RdYlBu_r",
    rasterized=True,
)

# =====

# alternative (not using xarray)
# create norm & cmap to plot levels
# cmap, norm = mpu.from_levels_and_cmap(levels, "RdYlBu_r")

# h = ax.pcolormesh(
#     ds.longitude,
#     ds.latitude,
#     ds.RPSS,
#     transform=ccrs.PlateCarree(),
#     cmap=cmap,
#     norm=norm,
#     rasterized=True,
# )

# =====

cbar = mpu.colorbar(h, ax, extendfrac=0.1, aspect=15)

levels = [0, 0.5, 1]
hatches = ["", "..."]

signif_cyclic.plot.contourf(
    ax=ax,
    levels=levels,
    hatches=hatches,
    colors="none",
    transform=ccrs.PlateCarree(),
    add_colorbar=False,
)

ax.set_title("Step 5: plot the actual data")

# ax.set_title('T2M: Ranked probability skill score, JJA')

f.canvas.draw()
# plt.savefig('ex2_5_stippling_RPSS.png', dpi=300)

## Exercise: Show significant precipitation change

The precipitation data from CMIP5 contains p-values, indicating where the change between the historical and future period is significant (note: this is derived from a t-test, which is not necessarily appropriate for the data).

### Step 1

* Use contourf to plot the p-values

> Note: it's p-values and not a boolean indicating significance as above.

In [ ]:
# get data
lon, lat, pval = pr.lon, pr.lat, pr.pval
PVAL, LON = cutil.add_cyclic_point(pval, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot p-values here
# h = ax.contourf(...)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval = pr.lon, pr.lat, pr.pval
PVAL, LON = cutil.add_cyclic_point(pval, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot p-values here
h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree())

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

### Exercise - Step 2 & 3

* We want to stipple everything with a p-value smaller than 0.1: determine the levels
* Add the hatches

In [ ]:
# get data
lon, lat, pval = pr.lon, pr.lat, pr.pval
PVAL, LON = cutil.add_cyclic_point(pval, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# set the levels
# levels =

# add hatches
# hatches = [...]

h = ax.contourf(LON, lat, PVAL, transform=ccrs.PlateCarree())

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval = pr.lon, pr.lat, pr.pval
PVAL, LON = cutil.add_cyclic_point(pval, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ["..", ""]

h = ax.contourf(
    LON, lat, PVAL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches
)

ax.set_global()

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

### Exercise - Step 4 & 5

* Remove the colors with `colors="none"`
* Plot the relative precipitation change below the hatches
* Make sure the right data is used for the colorbar

In [ ]:
# get data
lon, lat, pval, pr_rel = pr.lon, pr.lat, pr.pval, pr.pr_rel

PVAL, LON = cutil.add_cyclic_point(pval, lon)
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot relative precipitation change
# h = ax.contourf(...)

# plot significance stippling
# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ["...", ""]
h = ax.contourf(
    LON, lat, PVAL, transform=ccrs.PlateCarree(), levels=levels, hatches=hatches
)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

### Solution

In [ ]:
# get data
lon, lat, pval, pr_rel = pr.lon, pr.lat, pr.pval, pr.pr_rel

PVAL, LON = cutil.add_cyclic_point(pval, lon)
PR_REL, LON = cutil.add_cyclic_point(pr_rel, lon)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot relative precipitation change
# set levels
levels = np.arange(-20, 101, 20)
h = ax.contourf(
    LON, lat, PR_REL, transform=ccrs.PlateCarree(), levels=levels, extend="both"
)

# plot significance stippling
# set the levels
levels = [0, 0.1, 1]
# add hatches
hatches = ["...", ""]
ax.contourf(
    LON,
    lat,
    PVAL,
    transform=ccrs.PlateCarree(),
    levels=levels,
    hatches=hatches,
    colors="none",
)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

plt.draw()

## Manual hatches

You can also manually add hatches by adding points at the coordinates of all grid cells that are significant. This can be considerably faster than with using hatches, however, it leads to a hatching pattern that depends on the resolution of the data which is generally not desirable.

In [ ]:
# get data
pr_rel_cyclic = mpu.cyclic_dataarray(pr.pr_rel)

# ====

# plot
f, ax = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Robinson()))
ax.coastlines()

# plot relative precipitation change
# set levels
levels = np.arange(-20, 101, 20)
h = pr_rel_cyclic.plot.contourf(
    ax=ax,
    transform=ccrs.PlateCarree(),
    levels=levels,
    extend="both",
    add_colorbar=False,
    cmap="viridis",
)

# 'manual' stippling

# create array with all lat/ lon combinations
LON, LAT = np.meshgrid(pr.lon, pr.lat)

# find significant points (need to use .values)
sig = pr.pval.values.flatten() <= 0.1

LON = LON.flatten()[sig == 1]
LAT = LAT.flatten()[sig == 1]

# add scatterpoints
ax.plot(LON, LAT, ".", color="0.1", transform=ccrs.PlateCarree(), ms=1)

# add colorbar
mpu.colorbar(h, ax, spacing="proportional")

f.canvas.draw()